<a href="https://colab.research.google.com/github/Nataliia-Shap/the-one/blob/main/pre_trainedvgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.xception import Xception 
from keras.applications.xception import preprocess_input
from keras.applications.xception import decode_predictions
base_model = Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#model.compile(optimizer=keras.optimizers.Adam(),
#              loss=keras.losses.BinaryCrossentropy(from_logits=True),
#              metrics=[keras.metrics.BinaryAccuracy()])

#model.fit(new_dataset, epochs=10, callbacks=..., validation_data=...)

batch_size = 6 #16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/train',  # this is the target directory 'data/train'
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/validation',#'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
model.fit_generator(
        train_generator,
        steps_per_epoch=800 // batch_size, #2000
        epochs=5,#50
        validation_data=validation_generator,
        validation_steps=300 // batch_size) #800
model.save_weights('first_try.h5')  # always save your weights after training or during training

Found 3821 images belonging to 2 classes.
Found 399 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
133/133 [==============================] - 417s 3s/step - loss: 0.1343 - accuracy: 0.9799 - val_loss: 0.0632 - val_accuracy: 0.9900
Epoch 2/5
133/133 [==============================] - 300s 2s/step - loss: 0.0632 - accuracy: 0.9950 - val_loss: 0.1116 - val_accuracy: 0.9867
Epoch 3/5
133/133 [==============================] - 271s 2s/step - loss: 0.0469 - accuracy: 0.9950 - val_loss: 0.0573 - val_accuracy: 0.9933
Epoch 4/5
133/133 [==============================] - 224s 2s/step - loss: 0.0234 - accuracy: 0.9962 - val_loss: 9.4403e-04 - val_accuracy: 1.0000
Epoch 5/5
133/133 [==============================] - 176s 1s/step - loss: 0.0436 - accuracy: 0.9950 - val_loss: 0.0526 - val_accuracy: 0.9967
